In [2]:
from dataprocess import train_loader
import torch

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

FileNotFoundError: [Errno 2] No such file or directory: './data/train.json'

In [2]:
# 定义模型
class SVHN_Model1(nn.Module):
    def __init__(self):
        super(SVHN_Model1,self).__init__()
        # CNN特征提取模块
        self.cnn = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=(3,3),stride=(2,2)),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,32,kernel_size=(3,3),stride=(2,2)),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        # 6个全连接层分类
        self.fc1 = nn.Linear(32*3*7,11)
        self.fc2 = nn.Linear(32*3*7,11)
        self.fc3 = nn.Linear(32*3*7,11)
        self.fc4 = nn.Linear(32*3*7,11)
        self.fc5 = nn.Linear(32*3*7,11)
        self.fc6 = nn.Linear(32*3*7,11)

    def forward(self,x):  
        feat = self.cnn(x)
        # 全连接之前需要将之前的数据拉平
        feat = feat.view(feat.shape[0],-1)
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        c5 = self.fc5(feat)
        c6 = self.fc6(feat)
        return c1,c2,c3,c4,c5,c6

model = SVHN_Model1()

In [3]:
model

SVHN_Model1(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=672, out_features=11, bias=True)
  (fc2): Linear(in_features=672, out_features=11, bias=True)
  (fc3): Linear(in_features=672, out_features=11, bias=True)
  (fc4): Linear(in_features=672, out_features=11, bias=True)
  (fc5): Linear(in_features=672, out_features=11, bias=True)
  (fc6): Linear(in_features=672, out_features=11, bias=True)
)

In [4]:
# 学习率
lr = 0.005
epochs = 10
# 定义损失
criterion = nn.CrossEntropyLoss()
# 定义优化器
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

loss_plot = []  # 记录损失
c0_plot = []  # 记录正确率
# 训练
for epoch in range(epochs):
    for data in train_loader:
        c0,c1,c2,c3,c4,c5 = model(data[0])  # 第一个图的6个位置的模型输出概率[p1，p2，p3，p4，p5，p6]
        # 计算每一个字符的损失，并相加得处总损失，再求平均
        # criterion参数（模型输出结果，真实标签），以此求出损失
        loss = criterion(c0, data[1][:, 0]) + \
                criterion(c1, data[1][:, 1]) + \
                criterion(c2, data[1][:, 2]) + \
                criterion(c3, data[1][:, 3]) + \
                criterion(c4, data[1][:, 4]) + \
                criterion(c5, data[1][:, 5])
        loss /= 6
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_plot.append(loss.item())
        c0_plot.append((c0.argmax(1) == data[1][:,0]).sum().item()*1.0 / c0.shape[0])
    print(epoch)
            


NameError: name 'train_loader' is not defined